In [ ]:
from gpsprocessor import gps_data_utils
from shapely.geometry import Point
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster

### Set Depot Points, Boundary and Buffer here

In [ ]:
p1 = Point(19.4230387681323, -99.1283375653607)
p2 = Point(19.4226931599795, -99.1282858845984)
p3 = Point(19.4223342584274, -99.128116747558)
p4 = Point(19.4215765747699, -99.1277173962126)
p5 = Point(19.4212024412726, -99.1275371303434)
p6 = Point(19.4204750900101, -99.1271367956332)
p7 = Point(19.4203918053066, -99.1270190501302)
p8 = Point(19.4174929683974, -99.1274678598945)
p9 = Point(19.4172999746193, -99.1275045888836)
p10 = Point(19.4176612061906, -99.1318657926695)
p11 = Point(19.4179003490392, -99.1319983339035)
p12 = Point(19.4237439344618, -99.1310950549938)
p13 = Point(19.4233598470446, -99.1283536711016)

points = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12,p13,p1]
depot = gps_data_utils.set_depot_boundary(points,buffer = 0)

### Read and Validate GPS Data here

In [ ]:
'''
error_factor here is used as 1.5, i.e., 
    relative_error = abs(velocity_gps-velocity_vehicle) / velocity_vehicle 
    =>  ( 5 - 1 ) / 1 > 1.5
    => Calculated GPS speed is way higher than observed speed of the vehicle, i.e. gps walking
    => higher error_factor will allow more GPS walking
'''
error_factor = 1.25

In [ ]:
cols = ['UnityLicensePlate', 'RowReferenceTime', 'lat', 'lon','Speed','EngineStatus']
date_col = ['RowReferenceTime']
d_format = 'ISO8601'
file = 'C:/Users/achad/OneDrive - Microgrid Labs/Desktop/fornow/trip_data_10Units_1Year_V2.csv'
df = gps_data_utils.read_data(file,cols,date_col,d_format)

In [ ]:
df = gps_data_utils.validate_mandatory_cols(df,'RowReferenceTime','lat','lon','Speed','UnityLicensePlate')

In [ ]:
df = gps_data_utils.add_date_time_month_df(df)

In [ ]:
df = df.groupby(by=['vehicleid','date']).apply(gps_data_utils.add_prev_latlong).reset_index(drop=True)

In [ ]:
df = gps_data_utils.check_veh_within_depot(df,depot)

In [ ]:
df = gps_data_utils.calculate_speed_flag(df, error_factor)

In [ ]:
df = gps_data_utils.blank_filter(df)

In [ ]:
gdf = gps_data_utils.df_to_gdf(df)
tolerance = 0.05 
simplified_lines = gdf['geometry'].simplify(tolerance)
simplified_gdf = gpd.GeoDataFrame(geometry=simplified_lines)

In [ ]:
maps = folium.Map(max_zoom=24)
fg = MarkerCluster().add_to(maps)
for _, row in simplified_gdf.iterrows():
    lon, lat = row['geometry'].x, row['geometry'].y
    folium.Marker(location=[lat, lon],popup=f"Index:{row.name}").add_to(fg)

In [ ]:
maps.fit_bounds(fg.get_bounds())
maps